In [ ]:
# Ensure we use OTEL tracing.

import os

os.environ["TRULENS_OTEL_TRACING"] = "1"

In [ ]:
# Set up python resolution paths.

from pathlib import Path
import sys

# Add base dir to path to be able to access test folder.
base_dir = Path().cwd().parent.parent.resolve()
if str(base_dir) not in sys.path:
    print(f"Adding {base_dir} to sys.path")
    sys.path.append(str(base_dir))

In [ ]:
# Set up logging.

import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
handler.addFilter(logging.Filter("trulens"))
handler.setFormatter(formatter)
root.addHandler(handler)

In [ ]:
# Create snowpark session.

from snowflake.snowpark import Session

snowflake_connection_parameters = {
    "account": os.environ["SNOWFLAKE_ACCOUNT"],
    "user": os.environ["SNOWFLAKE_USER"],
    "password": os.environ["SNOWFLAKE_USER_PASSWORD"],
    "database": os.environ["SNOWFLAKE_DATABASE"],
    "schema": os.environ["SNOWFLAKE_SCHEMA"],
    "role": os.environ["SNOWFLAKE_ROLE"],
    "warehouse": os.environ["SNOWFLAKE_WAREHOUSE"],
}
snowpark_session = Session.builder.configs(
    snowflake_connection_parameters
).create()

In [ ]:
# Create TruSession.

from trulens.connectors.snowflake import SnowflakeConnector
from trulens.core.session import TruSession

tru_session = TruSession(
    connector=SnowflakeConnector(snowpark_session=snowpark_session)
)

In [ ]:
# Define app.

from trulens.core.otel.instrument import instrument
from trulens.otel.semconv.trace import SpanAttributes


class TestApp:
    @instrument(span_type=SpanAttributes.SpanType.MAIN)
    def respond_to_query(self, query: str) -> str:
        return f"answer: {self.nested(query)}"

    @instrument(attributes={"nested_attr1": "value1"})
    def nested(self, query: str) -> str:
        return f"nested: {self.nested2(query)}"

    @instrument(
        attributes=lambda ret, exception, *args, **kwargs: {
            "nested2_ret": ret,
            "nested2_args[1]": args[1],
        }
    )
    def nested2(self, query: str) -> str:
        nested_result = ""

        try:
            nested_result = self.nested3(query)
        except Exception:
            pass

        return f"nested2: {nested_result}"

    @instrument(
        attributes=lambda ret, exception, *args, **kwargs: {
            "nested3_ex": exception.args if exception else None,
            "nested3_ret": ret,
            "selector_name": "special",
            "cows": "moo",
        }
    )
    def nested3(self, query: str) -> str:
        if query == "throw":
            raise ValueError("nested3 exception")
        return "nested3"

In [ ]:
# Create TruLens instrumented app from custom app.

from datetime import datetime

from trulens.apps.app import TruApp

APP_NAME = f"{os.getlogin()}'s test app"
APP_VERSION = str(datetime.now())

test_app = TestApp()
tru_app = TruApp(
    test_app,
    app_name=APP_NAME,
    app_version=APP_VERSION,
    main_method=test_app.respond_to_query,
)

In [ ]:
# Run app directly.

with tru_app(run_name="test_run", input_id="456") as recording:
    test_app.respond_to_query("test")

with tru_app() as recording:
    test_app.respond_to_query("throw")

# Without flushing, the spans are not guaranteed to be sent.
tru_session.force_flush()

In [ ]:
# Run the app on a pandas DataFrame.

import pandas as pd
from trulens.core.app import App


def run_on_df(
    tru_app: App,
    run_name: str,
    inputs: pd.DataFrame,
    input_id_column: str,
    input_column: str,
):
    for _, row in inputs.iterrows():
        with tru_app(run_name=run_name, input_id=row[input_id_column]):
            main_method = getattr(test_app, tru_app.main_method_name)
            main_method(row[input_column])
    tru_app.session.force_flush()


run_on_df(
    tru_app,
    "df_run",
    pd.DataFrame({"input_id": [1, 2], "input": ["a", "b"]}),
    "input_id",
    "input",
)

In [ ]:
# Read the event table.

import time


def wait_for_nonzero_results(
    num_retries: int = 20, retry_cooldown_in_seconds: int = 5
):
    q = """
        SELECT
            *
        FROM
            table(snowflake.local.GET_AI_OBSERVABILITY_EVENTS(
                ?,
                ?,
                ?,
                'external agent'
            ))
        """
    for _ in range(num_retries):
        ret = snowpark_session.sql(
            q,
            params=[
                snowpark_session.get_current_database(),
                snowpark_session.get_current_schema(),
                APP_NAME,
            ],
        ).to_pandas()
        if len(ret) > 0:
            return ret
        time.sleep(retry_cooldown_in_seconds)
    raise ValueError("No results found!")


res = wait_for_nonzero_results()
res